In [2]:
from concurrent.futures import ThreadPoolExecutor
import time
import pandas as pd
pd.set_option('display.max_colwidth', 500)


In [ ]:
%set_env ANTHROPIC_API_KEY=${ANTHROPIC_API_KEY}%set_env LANGSMITH_TRACING=true%set_env LANGSMITH_API_KEY=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%set_env TAVILY_API_KEY=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%set_env OPENAI_API_KEY=${OPENAI_API_KEY}%set_env LANGCHAIN_API_KEY=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%set_env LANGCHAIN_TRACING_V2=true%set_env LANGCHAIN_PROJECT="Multi-agent Collaboration"

In [4]:
import configfrom openai import OpenAI# llm_api_key = config.CUSTOM_API_KEY"https://api.platform.a15t.com/v1"client = OpenAI(    api_key = llm_api_key,    base_url = llm_api_url,    # max_tokens=1000)# from langchain.chat_models import ChatOpenAIfrom langchain_openai import ChatOpenAIfrom langchain_anthropic import ChatAnthropicfrom langchain.schema import AIMessage, HumanMessage, SystemMessageimport pandas as pddef ChatAnthropicSKT(model="skt/claude-3-5-sonnet-20241022", max_tokens=100):    

In [5]:
from typing import List, Tuple, Union, Dict, Anyimport astimport reimport jsonimport globdef dataframe_to_markdown_prompt(df, max_rows=None):    # Limit rows if needed    if max_rows is not None and len(df) > max_rows:        display_df = df.head(max_rows)        truncation_note = f"\n[Note: Only showing first {max_rows} of {len(df)} rows]"    else:        display_df = df        truncation_note = ""        # Convert to markdown    df_markdown = display_df.to_markdown()        prompt = f"""    {df_markdown}    {truncation_note}    """        return promptdef replace_strings(text, replacements):    for old, new in replacements.items():        text = text.replace(old, new)            return textdef clean_segment(segment):    """    Given a segment that is expected to be quoted (i.e. begins and ends with    the same single or double quote), remove any occurrences of that quote    from the inner content.    For example, if segment is:         "에이닷 T 멤버십 쿠폰함에 "에이닷은통화요약된닷" 입력"    then the outer quotes are preserved but the inner double quotes are removed.    """    segment = segment.strip()    if len(segment) >= 2 and segment[0] in ['"', "'"] and segment[-1] == segment[0]:        q = segment[0]        # Remove inner occurrences of the quote character.        inner = segment[1:-1].replace(q, '')        return q + inner + q    return segmentdef split_key_value(text):    """    Splits text into key and value based on the first colon that appears    outside any quoted region.    If no colon is found outside quotes, the value will be returned empty.    """    in_quote = False    quote_char = ''    for i, char in enumerate(text):        if char in ['"', "'"]:            # Toggle quote state (assumes well-formed starting/ending quotes for each token)            if in_quote:                if char == quote_char:                    in_quote = False                    quote_char = ''            else:                in_quote = True                quote_char = char        elif char == ':' and not in_quote:            return text[:i], text[i+1:]    return text, ''def split_outside_quotes(text, delimiter=','):    """    Splits the input text on the given delimiter (default comma) but only    if the delimiter occurs outside of quoted segments.    Returns a list of parts.    """    parts = []    current = []    in_quote = False    quote_char = ''    for char in text:        if char in ['"', "'"]:            # When encountering a quote, toggle our state            if in_quote:                if char == quote_char:                    in_quote = False                    quote_char = ''            else:                in_quote = True                quote_char = char            current.append(char)        elif char == delimiter and not in_quote:            parts.append(''.join(current).strip())            current = []        else:            current.append(char)    if current:        parts.append(''.join(current).strip())    return partsdef clean_ill_structured_json(text):    """    Given a string that is intended to represent a JSON-like structure    but may be ill-formed (for example, it might contain nested quotes that    break standard JSON rules), attempt to “clean” it by processing each    key–value pair.        The function uses the following heuristics:      1. Split the input text into comma-separated parts (only splitting         when the comma is not inside a quoted string).      2. For each part, split on the first colon (that is outside quotes) to separate key and value.      3. For any segment that begins and ends with a quote, remove any inner occurrences         of that same quote.      4. Rejoin the cleaned key and value.        Note: This approach does not build a fully robust JSON parser. For very complex          or deeply nested ill-structured inputs further refinement would be needed.    """    # First, split the text by commas outside of quotes.    parts = split_outside_quotes(text, delimiter=',')        cleaned_parts = []    for part in parts:        # Try to split into key and value on the first colon not inside quotes.        key, value = split_key_value(part)        key_clean = clean_segment(key)        value_clean = clean_segment(value) if value.strip() != "" else ""        if value_clean:            cleaned_parts.append(f"{key_clean}: {value_clean}")        else:            cleaned_parts.append(key_clean)        # Rejoin the cleaned parts with commas (or you can use another format if desired)    return ', '.join(cleaned_parts)def repair_json(broken_json):        # json_str = broken_json.replace("'",'"')        # Fix unquoted values (like NI00001863)    json_str = re.sub(r':\s*([a-zA-Z0-9_]+)(\s*[,}])', r': "\1"\2', broken_json)        # Fix unquoted keys    json_str = re.sub(r'([{,])\s*([a-zA-Z0-9_]+):', r'\1 "\2":', json_str)        # Fix trailing commas    json_str = re.sub(r',\s*}', '}', json_str)        return json_strdef extract_json_objects(text):    # More sophisticated pattern that tries to match proper JSON syntax    pattern = r'(\{(?:[^{}]|(?:\{(?:[^{}]|(?:\{[^{}]*\}))*\}))*\})'        result = []    for match in re.finditer(pattern, text):        potential_json = match.group(0)        try:            # Try to parse and validate            # json_obj = json.loads(repair_json(potential_json))            json_obj = ast.literal_eval(clean_ill_structured_json(repair_json(potential_json)))            result.append(json_obj)        except json.JSONDecodeError:            # Not valid JSON, skip            pass        return resultdef extract_between(text, start_marker, end_marker):    start_index = text.find(start_marker)    if start_index == -1:        return None        start_index += len(start_marker)    end_index = text.find(end_marker, start_index)    if end_index == -1:        return None        return text[start_index:end_index]def extract_content(text: str, tag_name: str) -> List[str]:    pattern = f'<{tag_name}>(.*?)</{tag_name}>'    matches = re.findall(pattern, text, re.DOTALL)    return matchesdef clean_bad_text(text):    import re        if not isinstance(text, str):        return ""        # Remove URLs and emails    text = re.sub(r'https?://\S+|www\.\S+', ' ', text)    text = re.sub(r'\S+@\S+', ' ', text)        # Keep Korean, alphanumeric, spaces, and specific punctuation    text = re.sub(r'[^\uAC00-\uD7A3\u1100-\u11FF\w\s\.\?!,]', ' ', text)        # Normalize whitespace    text = re.sub(r'\s+', ' ', text).strip()        return textdef clean_text(text):    """    Cleans text by removing special characters that don't affect fine-tuning.    Preserves important structural elements like quotes, brackets, and JSON syntax.    Specifically handles Korean text (Hangul) properly.        Args:        text (str): The input text to clean            Returns:        str: Cleaned text ready for fine-tuning    """    import re        # Preserve the basic structure by temporarily replacing important characters    # with placeholder tokens that won't be affected by cleanup        # Step 1: Temporarily replace JSON structural elements    placeholders = {        '"': "DQUOTE_TOKEN",        "'": "SQUOTE_TOKEN",        "{": "OCURLY_TOKEN",        "}": "CCURLY_TOKEN",        "[": "OSQUARE_TOKEN",        "]": "CSQUARE_TOKEN",        ":": "COLON_TOKEN",        ",": "COMMA_TOKEN"    }        for char, placeholder in placeholders.items():        text = text.replace(char, placeholder)        # Step 2: Remove problematic characters        # Remove control characters (except newlines, carriage returns, and tabs which can be meaningful)    text = re.sub(r'[\x00-\x08\x0B\x0C\x0E-\x1F\x7F]', '', text)        # Normalize all types of newlines to \n    text = re.sub(r'\r\n|\r', '\n', text)        # Remove zero-width characters and other invisible unicode    text = re.sub(r'[\u200B-\u200D\uFEFF\u00A0]', '', text)        # MODIFIED: Keep Korean characters (Hangul) along with other useful character sets    # This regex keeps:    # - ASCII (Basic Latin): \x00-\x7F    # - Latin-1 Supplement: \u0080-\u00FF    # - Latin Extended A & B: \u0100-\u017F\u0180-\u024F    # - Greek and Coptic: \u0370-\u03FF    # - Cyrillic: \u0400-\u04FF    # - Korean Hangul Syllables: \uAC00-\uD7A3    # - Hangul Jamo (Korean alphabet): \u1100-\u11FF    # - Hangul Jamo Extended-A: \u3130-\u318F    # - Hangul Jamo Extended-B: \uA960-\uA97F    # - Hangul Compatibility Jamo: \u3130-\u318F    # - CJK symbols and punctuation: \u3000-\u303F    # - Full-width forms (often used with CJK): \uFF00-\uFFEF    # - CJK Unified Ideographs (Basic common Chinese/Japanese characters): \u4E00-\u9FFF        # Instead of removing characters, we'll define which ones to keep    allowed_chars_pattern = r'[^\x00-\x7F\u0080-\u00FF\u0100-\u024F\u0370-\u03FF\u0400-\u04FF' + \                           r'\u1100-\u11FF\u3130-\u318F\uA960-\uA97F\u3000-\u303F' + \                           r'\uAC00-\uD7A3\uFF00-\uFFEF\u4E00-\u9FFF\n\r\t ]'    text = re.sub(allowed_chars_pattern, '', text)        # Step 3: Normalize whitespace (but preserve deliberate line breaks)    text = re.sub(r'[ \t]+', ' ', text)  # Convert multiple spaces/tabs to single space        # First ensure all newlines are standardized    text = re.sub(r'\r\n|\r', '\n', text)  # Convert all newline variants to \n        # Then normalize multiple blank lines to at most two    text = re.sub(r'\n\s*\n+', '\n\n', text)  # Convert multiple newlines to at most two        # Step 4: Restore original JSON structural elements    for char, placeholder in placeholders.items():        text = text.replace(placeholder, char)        # Step 5: Fix common JSON syntax issues that might remain    # Fix spaces between quotes and colons in JSON    text = re.sub(r'"\s+:', r'":', text)        # Fix trailing commas in arrays    text = re.sub(r',\s*]', r']', text)        # Fix trailing commas in objects    text = re.sub(r',\s*}', r'}', text)        return textdef remove_control_characters(text):    if isinstance(text, str):        # Remove control characters except commonly used whitespace        return re.sub(r'[\x00-\x08\x0B-\x0C\x0E-\x1F\x7F]', '', text)    return textimport openaifrom langchain.chains import RetrievalQAfrom langchain.llms.openai import OpenAIChat  # For compatibility with newer setup# Create a custom LLM class that uses the OpenAI client directlyclass CustomOpenAI:    def __init__(self, model="skt/a.x-3-lg"):        self.model = model            def __call__(self, prompt):        response = client.chat.completions.create(            model=self.model,            messages=[                {"role": "system", "content": "You are a helpful assistant."},                {"role": "user", "content": prompt}            ],            temperature=0.1        )        return response.choices[0].message.content# Create a simple retrieval functiondef get_relevant_context(query, vectorstore, topk=5):    docs = vectorstore.similarity_search(query, k=topk)    context = "\n\n".join([doc.page_content for doc in docs])    titles = ", ".join(set([doc.metadata['title'] for doc in docs if 'title' in doc.metadata.keys()]))    return {'title':titles, 'context':context}# Create a function to combine everythingdef answer_question(query, vectorstore):    # Get relevant context    context = get_relevant_context(query, vectorstore)        # Create combined prompt    prompt = f"Answer the following question based on the provided context:\n\nContext: {context}\n\nQuestion: {query}\n\nAnswer:"        # Use OpenAI directly    custom_llm = CustomOpenAI()  # Or your preferred model    response = custom_llm(prompt)        return responsedef is_list_of_dicts(var):    # Check if the variable is a list    if not isinstance(var, list):        return False        # Check if the list is not empty and all elements are dictionaries    if not var:  # Empty list        return False            # Check that all elements are dictionaries    return all(isinstance(item, dict) for item in var)def remove_duplicate_dicts(dict_list):    result = []    seen = set()    for d in dict_list:        # Convert dictionary to a hashable tuple of items        t = tuple(sorted(d.items()))        if t not in seen:            seen.add(t)            result.append(d)    return result

In [6]:
from rapidfuzz import fuzz, process
import re
class KoreanEntityMatcher:
    def __init__(self, min_similarity=75, ngram_size=2, min_entity_length=2):
        self.min_similarity = min_similarity
        self.ngram_size = ngram_size
        self.min_entity_length = min_entity_length
        self.entities = []
        self.entity_data = {}
    def build_from_list(self, entities):
        """Build entity index from a list of entities"""
        self.entities = []
        self.entity_data = {}
        for i, entity in enumerate(entities):
            if isinstance(entity, tuple) and len(entity) == 2:
                entity_name, data = entity
                self.entities.append(entity_name)
                self.entity_data[entity_name] = data
            else:
                self.entities.append(entity)
                self.entity_data[entity] = {'id': i, 'entity': entity}
        # Create n-gram index for faster candidate selection
        self._build_ngram_index(n=self.ngram_size)
    def _build_ngram_index(self, n=2):
        """Build n-gram index optimized for Korean characters"""
        self.ngram_index = {}
        for entity in self.entities:
            # Skip entities shorter than min_entity_length
            if len(entity) < self.min_entity_length:
                continue
            # Create n-grams for the entity
            entity_chars = list(entity)  # Split into characters for proper Korean handling
            ngrams = []
            # Create character-level n-grams (better for Korean)
            for i in range(len(entity_chars) - n + 1):
                ngram = ''.join(entity_chars[i:i+n])
                ngrams.append(ngram)
            # Add entity to the index for each n-gram
            for ngram in ngrams:
                if ngram not in self.ngram_index:
                    self.ngram_index[ngram] = set()
                self.ngram_index[ngram].add(entity)
    def _get_candidates(self, text, n=None):
        """Get candidate entities based on n-gram overlap (optimized for Korean)"""
        if n is None:
            n = self.ngram_size
        text_chars = list(text)  # Split into characters for proper Korean handling
        text_ngrams = set()
        # Create character-level n-grams
        for i in range(len(text_chars) - n + 1):
            ngram = ''.join(text_chars[i:i+n])
            text_ngrams.add(ngram)
        candidates = set()
        for ngram in text_ngrams:
            if ngram in self.ngram_index:
                candidates.update(self.ngram_index[ngram])
        # Prioritize candidates with multiple n-gram matches
        candidate_scores = {}
        for candidate in candidates:
            candidate_chars = list(candidate)
            candidate_ngrams = set()
            for i in range(len(candidate_chars) - n + 1):
                ngram = ''.join(candidate_chars[i:i+n])
                candidate_ngrams.add(ngram)
            overlap = len(candidate_ngrams.intersection(text_ngrams))
            candidate_scores[candidate] = overlap
        # Return candidates sorted by n-gram overlap score
        return sorted(candidate_scores.items(), key=lambda x: x[1], reverse=True)
    def find_entities(self, text, max_candidates_per_span=10):
        """Find entity matches in Korean text using fuzzy matching"""
        # Extract spans that might contain entities
        potential_spans = self._extract_korean_spans(text)
        matches = []
        for span_text, start, end in potential_spans:
            if len(span_text.strip()) < self.min_entity_length:  # Skip spans shorter than min_entity_length
                continue
            # Get candidate entities based on n-gram overlap
            candidates = self._get_candidates(span_text)
            # If no candidates found through n-gram filtering, skip
            if not candidates:
                continue
            # Limit the number of candidates to check
            top_candidates = [c[0] for c in candidates[:max_candidates_per_span]]
            # Find best fuzzy matches
            best_matches = process.extract(
                span_text,
                top_candidates,
                scorer=fuzz.ratio,  # Better for Korean than token_sort_ratio
                score_cutoff=self.min_similarity,
                limit=3
            )
            for entity, score, _ in best_matches:
                matches.append({
                    'text': span_text,
                    'matched_entity': entity,
                    'score': score,
                    'start': start,
                    'end': end,
                    'data': self.entity_data.get(entity, {})
                })
        # Sort by position in text
        matches.sort(key=lambda x: (x['start'], -x['score']))
        # Handle overlapping matches by keeping the best match
        final_matches = self._resolve_overlapping_matches(matches)
        return final_matches
    def _extract_korean_spans(self, text):
        """Extract potential text spans that might be entities in Korean text"""
        spans = []
        min_len = self.min_entity_length
        # Korean text segmentation patterns
        # 1. Extract spans separated by common Korean delimiters
        for span in re.split(r'[,\.!?;:"\'\(\)\[\]\{\}\s]+', text):
            if span and len(span) >= min_len:
                span_pos = text.find(span)
                if span_pos != -1:
                    spans.append((span, span_pos, span_pos + len(span)))
        # 2. Extract noun phrases (sequences without particles/endings)
        # Korean particles often follow nouns, so look for sequences without typical particles
        for match in re.finditer(r'[가-힣a-zA-Z0-9]+(?:[^\s.,!?;:은는이가을를에서의로와과]\s*[가-힣a-zA-Z0-9]+)*', text):
            if len(match.group(0)) >= min_len:
                spans.append((match.group(0), match.start(), match.end()))
        # 3. Extract sequences of Korean with numbers and English (common for product names, etc.)
        for match in re.finditer(r'[가-힣a-zA-Z]+[\s\-]?[0-9]+(?:[\s\-][가-힣a-zA-Z0-9]+)*', text):
            if len(match.group(0)) >= min_len:
                spans.append((match.group(0), match.start(), match.end()))
        # 4. Extract fixed-length windows (useful for Korean where spaces are sometimes omitted)
        words = list(text)  # Split into characters
        # Use window sizes starting from min_entity_length up to 8
        window_sizes = list(range(min_len, 9))
        for size in window_sizes:
            for i in range(len(words) - size + 1):
                window = ''.join(words[i:i+size])
                spans.append((window, i, i + size))
        return spans
    def _resolve_overlapping_matches(self, matches):
        """Resolve overlapping matches by keeping the highest scoring match"""
        if not matches:
            return []
        # Sort by start position and then by score (descending)
        sorted_matches = sorted(matches, key=lambda x: (x['start'], -x['score']))
        final_matches = [sorted_matches[0]]
        current_end = sorted_matches[0]['end']
        for match in sorted_matches[1:]:
            # If this match starts after the previous match ends, add it
            if match['start'] >= current_end:
                final_matches.append(match)
                current_end = match['end']
            # If this match overlaps but has a higher score, replace the previous match
            elif match['score'] > final_matches[-1]['score'] and match['start'] < current_end:
                final_matches[-1] = match
                current_end = match['end']
        return final_matches
def find_entities_in_text(text, entity_list, min_similarity=75, ngram_size=2, min_entity_length=2):
    """
    Find entity matches in text using fuzzy matching.
    Parameters:
    -----------
    text : str
        The text to search for entities
    entity_list : list
        List of entities to match against
    min_similarity : int, default=75
        Minimum similarity score (0-100) for fuzzy matching
    ngram_size : int, default=2
        Size of character n-grams to use for indexing (2 or 3 recommended for Korean)
    min_entity_length : int, default=2
        Minimum length of entities to consider (characters)
    Returns:
    --------
    list
        List of matched entities with position and metadata
    """
    matcher = KoreanEntityMatcher(
        min_similarity=min_similarity,
        ngram_size=ngram_size,
        min_entity_length=min_entity_length
    )
    matcher.build_from_list(entity_list)
    matches = matcher.find_entities(text)
    return matches
# Function to highlight entities in text
def highlight_entities(text, matches):
    marked_text = text
    offset = 0
    for match in sorted(matches, key=lambda x: x['start'], reverse=True):
        start = match['start'] + offset
        end = match['end'] + offset
        entity = match['matched_entity']
        score = match['score']
        marked_text = marked_text[:start] + f"[{marked_text[start:end]}→{entity} ({score}%)]" + marked_text[end:]
        offset += len(f"[→{entity} ({score}%)]") + 2
    return marked_text

In [7]:
from difflib import SequenceMatcherdef calculate_list_similarity(list1, list2):    # Handle cases where input might be a dictionary or other types    if isinstance(list1, dict):        list1 = [str(item) for item in list1.values()]    if isinstance(list2, dict):        list2 = [str(item) for item in list2.values()]        # Ensure lists contain strings    list1 = [str(item) for item in list1]    list2 = [str(item) for item in list2]        # Convert lists to sets for comparison    set1 = set(sorted(set(list1)))    set2 = set(sorted(set(list2)))        # Calculate Jaccard similarity    intersection = len(set1.intersection(set2))    union = len(set1.union(set2))        return intersection / union if union > 0 else 0def calculate_text_similarity(text1, text2):    return SequenceMatcher(None, str(text1), str(text2)).ratio()def calculate_message_info_similarity(minfo1, minfo2):    if not isinstance(minfo1, dict) or not isinstance(minfo2, dict):        return 0.0        sender_sim = calculate_text_similarity(minfo1.get('sender',''), minfo2.get('sender',''))    title_sim = calculate_text_similarity(minfo1.get('title',''), minfo2.get('title',''))    main_theme_sim = calculate_text_similarity(minfo1.get('main_theme',''), minfo2.get('main_theme',''))    period_sim = calculate_text_similarity(minfo1.get('period',''), minfo2.get('period',''))        return (sender_sim + title_sim + main_theme_sim + period_sim) / 4def calculate_metadata_similarity(minfo1, minfo2):    if not isinstance(minfo1, dict) or not isinstance(minfo2, dict):        return 0.0        campaign_id_sim = calculate_text_similarity(minfo1.get('campaign_id',''), minfo2.get('campaign_id',''))    message_type_sim = calculate_text_similarity(minfo1.get('message_type',''), minfo2.get('message_type',''))    target_response_sim = calculate_text_similarity(minfo1.get('target_response',''), minfo2.get('target_response',''))        success_metrics_similarity = calculate_list_similarity(        minfo1.get('success_metrics',[]),         minfo2.get('success_metrics',[])    )        return (campaign_id_sim + message_type_sim + target_response_sim + success_metrics_similarity) / 4def calculate_product_similarity(prod1, prod2):    if not isinstance(prod1, dict) or not isinstance(prod2, dict):        return 0.0        name_sim = calculate_text_similarity(prod1.get('name',''), prod2.get('name',''))    category_sim = calculate_text_similarity(prod1.get('category',''), prod2.get('category',''))    benefit_sim = calculate_text_similarity(prod1.get('benefit',''), prod2.get('benefit',''))    action_sim = calculate_text_similarity(prod1.get('action',''), prod2.get('action',''))    conditions_sim = calculate_text_similarity(prod1.get('conditions',''), prod2.get('conditions',''))        return (name_sim + category_sim + benefit_sim + action_sim + conditions_sim) / 5def calculate_channel_similarity(chan1, chan2):    if not isinstance(chan1, dict) or not isinstance(chan2, dict):        return 0.0        type_sim = calculate_text_similarity(chan1.get('type',''), chan2.get('type',''))    value_sim = calculate_text_similarity(chan1.get('value',''), chan2.get('value',''))    action_sim = calculate_text_similarity(chan1.get('action',''), chan2.get('action',''))    availability_sim = calculate_text_similarity(chan1.get('availability',''), chan2.get('availability',''))        return (type_sim + value_sim + action_sim + availability_sim) / 4def calculate_target_similarity(target1, target2):    if not isinstance(target1, list) or not isinstance(target2, list):        return 0.0        similarities = []    for t1 in target1:        for t2 in target2:            segment_sim = calculate_text_similarity(t1.get('segment',''), t2.get('segment',''))            characteristics_sim = calculate_text_similarity(t1.get('characteristics',''), t2.get('characteristics',''))            similarities.append((segment_sim + characteristics_sim) / 2)        return max(similarities) if similarities else 0.0def calculate_dictionary_similarity(dict1, dict2):    # Calculate similarities for each component    message_info_similarity = calculate_message_info_similarity(        dict1.get('message_info',{}),         dict2.get('message_info',{})    )        meta_similarity = calculate_metadata_similarity(        dict1.get('metadata',{}),         dict2.get('metadata',{})    )        purpose_similarity = calculate_list_similarity(        dict1.get('purpose',[]),         dict2.get('purpose',[])    )        target_similarity = calculate_target_similarity(        dict1.get('target',[]),         dict2.get('target',[])    )        # Calculate product similarity    product_similarities = []    products1 = dict1.get('product', [])    products2 = dict2.get('product', [])    for p1, p2 in zip(products1, products2):        product_similarities.append(calculate_product_similarity(p1, p2))    product_similarity = sum(product_similarities) / len(product_similarities) if product_similarities else 0.0        # Calculate channel similarity    channel_similarities = []    channels1 = dict1.get('channel', [])    channels2 = dict2.get('channel', [])    for c1, c2 in zip(channels1, channels2):        channel_similarities.append(calculate_channel_similarity(c1, c2))    channel_similarity = sum(channel_similarities) / len(channel_similarities) if channel_similarities else 0.0    # Calculate overall similarity    overall_similarity = (        message_info_similarity +         purpose_similarity +         target_similarity +         product_similarity +         channel_similarity +        meta_similarity    ) / 5        return {        'overall_similarity': overall_similarity,        'message_info_similarity': message_info_similarity,        'purpose_similarity': purpose_similarity,        'target_similarity': target_similarity,        'product_similarity': product_similarity,        'channel_similarity': channel_similarity,        'meta_similarity': meta_similarity    }

In [ ]:
# mms_pdf = pd.read_excel("./data/mms_data_250408.xlsx", engine="openpyxl")
mms_pdf = pd.read_pickle("./data/mms_data_250408.pkl")
mms_pdf['msg'] = mms_pdf['msg_nm']+"\n"+mms_pdf['mms_phrs']
mms_pdf = mms_pdf.groupby(["msg_nm","mms_phrs","msg"])['offer_dt'].min().reset_index(name="offer_dt")
mms_pdf = mms_pdf.reset_index()
mms_pdf = mms_pdf.astype('str')
mms_pdf.head(1)
# item_pdf = pd.read_pickle("/home/skinet/myfiles/tos_ace/data/item_info_250318.pkl")

In [9]:
# item_pdf_raw = pd.read_excel("./data/item_info_250401.xlsx", engine="openpyxl")
item_pdf_raw = pd.read_pickle("./data/item_info_250401.pkl")
item_pdf = item_pdf_raw.drop_duplicates(['item_nm','item_desc']).copy()
item_pdf['item_item'] = item_pdf['item_nm']+"\n"+item_pdf['item_desc']
item_pdf['item_nm_cl'] = item_pdf['item_nm'].apply(clean_text)
item_pdf['item_desc_cl'] = item_pdf['item_desc'].fillna('').astype(str).apply(clean_text)
item_pdf['item_item_cl'] = item_pdf['item_nm_cl']+"\n"+item_pdf['item_desc_cl']
entity_list = []
for row in item_pdf.to_dict('records'):
    entity_list.append((row['item_nm'], {'item_id':row['item_id'],'category':row['item_cate_ax'], 'description':row['item_desc'], 'create_dt':row['create_dt']}))


In [ ]:
import sysimport pysqlite3sys.modules["sqlite3"] = pysqlite3from langchain.schema import Documentfrom langchain.embeddings import OpenAIEmbeddingsfrom langchain_chroma import Chromafrom chromadb import Clientfrom chromadb.config import Settings# Get the collection and delete it if it existscollection_name="item_embeddings_collection"# client_chromadb = Client(Settings(#     persist_directory="./chroma_db",#     is_persistent=True# ))# try:#     client_chromadb.delete_collection(collection_name)#     print(f"Deleted collection: {collection_name}")# except Exception as e:#     print(f"Collection doesn't exist or couldn't be deleted: {e}")item_pdf_rag = item_pdf_raw.copy()for i in range(10):    item_pdf_rag[f'item_nm_{i+1}'] = item_pdf_rag['item_nm']    documents = []for i, row in item_pdf_rag.iterrows():    # You can format the content however makes sense for your data    content = f"Row {i}: " + ", ".join([f"{col}: {val}" for col, val in row.items()])    # Or more specifically formatted based on your needs    # content = f"Customer: {row['name']}, Purchase: {row['item']}, Amount: ${row['amount']}"    names =  f"{row['item_nm']}"        metadata = {"title":names, "source": "dataframe", "row_id": i}    doc = Document(page_content=content, metadata=metadata)    documents.append(doc)    # Create embeddings and vectorstore# embeddings = OpenAIEmbeddings(openai_api_key=llm_api_key, openai_api_base=llm_api_url)from langchain.embeddings import HuggingFaceEmbeddingsembeddings = HuggingFaceEmbeddings(model_name="snunlp/KR-SBERT-V40K-klueNLI-augSTS")vectorstore = Chroma.from_documents(    documents,     embeddings,    collection_name=collection_name)qa_chain = RetrievalQA.from_chain_type(    llm=llm_cld35,    chain_type="stuff",    retriever=vectorstore.as_retriever())

In [ ]:
prompt = f"""아래 광고 메세지의 구조 분석해라.분석의 목적은 메세지에서 구조화된 정보를 추출하기 위함이다.다양한 구조의 다른 메세지들도 분석할 예정인데, 이를 위한 prompt에 추가할 JSON이 아래와 같은 데, 개선점을 파악해서, 새로운 JSON을 추천해라.===광고 메세지 예시==={mms_pdf['msg'][:100]}===PROMPT에 넣을 JSON==={schema_cld}"""cot_list = []for row in few_shot_pdf.to_dict('records'):    prompt = f"""    당신은 캠페인 메시지에서 정보를 추출하는 전문가입니다. 다음과 같은 schema에 매칭되는 JSON 개체를 만들기 위한 Chain-of-Thought를 만들어주세요.    당신이 제공한 Chain-of-Thought는 다른 LLM이 유사한 광고 메세지에서 구조화된 정보를 추출하는 데 사용됩니다.    다른 LLM이 이 Chain-of-Thought를 사용하여 광고 메세지에서 구조화된 정보를 추출할 수 있도록, 예시 광고 메세지의 특징을 파악해야 하지만, 그렇다고 너무 의존적이어서는 안됩니다.     왜나허면, 동일한 메세지 분석에 사용되는 것이 아니기 때문입니다.    Chain-of-Thought는 한국어로 작성하고, 다른 LLM에 prompt로 바로 사용될 수 있도록 hain-of-Thought 결과만을 제공해 주세요.        ===광고 메세지 예시===    메세지 제목: {row['msg_head']}    메세지 내용: {row['msg_body']}    ===결과 Schema===    {schema_cld}    """    row['cot_chat'] = (llm_chat.invoke(prompt).content)    cot_list.append(row)    if len(cot_list) % 10 == 0:        print(f"Processed {len(cot_list)} rows")    # print((llm_chat.invoke(prompt).content))

In [12]:
from langchain_anthropic import ChatAnthropicfrom langchain_core.prompts import ChatPromptTemplatefrom langchain_core.output_parsers import JsonOutputParser# Define your schemaschema_cld = {    "properties": {        "message_info": {            "type": "object",            "properties": {                "title": {"type": "string", "description": "광고 제목"},                "main_theme": {"type": "string", "description": "광고의 주요 테마/주제"},                "period": {"type": "string", "description": "이벤트/프로모션 기간"}            }        },        "purpose": {"type": "array", "description": "[상품 가입 유도, 대리점 방문 유도, 웹/앱 접속 유도, 이벤트 응모 유도, 할인 혜택 안내, 쿠폰 제공 안내, 경품 제공 안내, 기타 정보 제공] 중에서 선택. 애매모호한 상황인 경우, 복수 선택 가능."},        "target": {            "type": "array",            "items": {                "type": "object",                "properties": {                    "segment": {"type": "string", "description": "타겟 고객층"},                    "characteristics": {"type": "string", "description": "해당 타겟의 특성"},                    "priority": {"type": "integer", "description": "타겟팅 우선순위"}                }            }        },        "product": {            "type": "array",            "items": {                "type": "object",                "properties": {                    "name": {"type": "string", "description": "광고하는 제품이나 서비스. 광고 메세지에 표현된 원본 그대로 추출해 주세요."},                    "category": {"type": "string", "description": "광고 상품의 카테고리"},                    "benefit": {                        "type": "object",                        "properties": {                            "name": {"type": "string", "description": "제공되는 혜택 이름"},                            "type": {"type": "string", "description": "혜택 타입. [할인, 쿠폰, 경품, 기타] 중에서 선택"}                        }                    },                    "conditions": {"type": "string", "description": "혜택/구매 조건"},                    "action": {"type": "string", "description": "해당 상품에 대해서 고객이 하기 바라는 행동. [구매, 가입, 사용, 방문, 참여, 작성, 기타] 중에서 선택"}                },                "required": ["name", "info"]            }        },        "channel": {            "type": "array",            "items": {                "type": "object",                "properties": {                    "type": {"type": "string", "description": "채널의 종류 (URL, 전화번호, 앱, 대리점)"},                    "value": {"type": "string", "description": "실제 URL, 전화번호, 앱 이름, 대리점 이름 등의 상세 정보"},                    "action": {"type": "string", "description": "채널 제공의 목적. [가입, 추가 정보, 문의, 수신, 수신 거부] 중에서 선택"},                    "primary": {"type": "boolean", "description": "주요 채널 여부"},                    "availability": {"type": "string", "description": "채널 이용 가능 시간/조건"},                    "benefit": {"type": "string", "description": "해당 채널 이용 시 혜택"}                },                "required": ["type", "value"]            }        }    },    "metadata": {        "type": "object",        "properties": {            "message_type": {"type": "string"},            "target_response": {"type": "string"},            "success_metrics": {"type": "array"}        }    },    "required": ["purpose", "target", "product", "channel","metadata"],    "objectType": "object"}# Create a prompt templateprompt = ChatPromptTemplate.from_messages([    ("system", "당신은 캠페인 메시지에서 정보를 추출하는 전문가입니다. 다음과 같은 schema에 매칭되는 JSON 개체를 만들어 주세요:\n {schema}\n\n제품 참조 정보:\n{rag_context}"),    ("human", "{text}")])# Create the parserparser = JsonOutputParser()# Create the chainextraction_chain = prompt | llm_cld37 | parser# Run the extractionmsg_text = """광고 제목: [SK텔레콤] ACT대리점 SKT 신현본점에서 꼭꼭 숨겨뒀던 할인 꿀팁 방출    광고 내용: (광고)[SKT] 고객님 안녕하세요._루원이편한세상 하늘채아파트 정문 신현본점에서_통신비 할인 비책 공개합니다!__① 기가인터넷+TV 요금무료_ 5월 한정 프로모션, 하나 원더카드가_ SK인터넷요금 2년간 지원_ (전월 카드 40만원이상 사용기준)__② S23 할인이 중복_ S23은 약정할인과 제휴카드 할인이 중복된다는 사실!_ 갓성비 갤럭시 퀀텀3도 마찬가지__③ SK공식인증매장에서 더 싸다?_ SK매직/ADT캡스와 제휴하여_ 같은 공기청정기/정수기/홈보안이라도 훨씬 저렴하게 이용!__□ 매장으로 문의주세요 (네이버,매장 홈페이지로 방문예약 가능)__◆ 단골등록 이벤트_ 단골고객 모두에게_ 스마트폰 구매 시 나와 지인 같이 할인받는_ "지인찬스 쿠폰" 4만원권을 발송__■ ACT대리점 신현본점_- 주소: 인천 서구 가정로 387, 가동 201호 SK텔레콤_- 연락처: 0507-1404-2560_- 네이버로 보기: http://t-mms.kr/t.do?m=#61&s=19725&a=&u=https://m.place.naver.com/place/1108219331/home_- 매장 위치보기: http://t-mms.kr/t.do?m=#61&s=19726&a=&u=https://m.tworld.co.kr/customer/agentsearch/detail?code=D13546-0297_- 매장 홈페이지: http://t-mms.kr/t.do?m=#61&s=19727&a=&u=https://tworldfriends.co.kr/D135460297__■ 문의 : SKT고객센터(1558,무료)_무료 수신거부 1504_""".replace('_','\n')# print(msg_text)# rag_context = get_relevant_context(msg_text, vectorstore, topk=20)['title']matches = find_entities_in_text(    msg_text,     entity_list,     min_similarity=70,  # Lower threshold for more matches    ngram_size=3,       # Bigrams work well for Korean    min_entity_length=3 # Only consider entities with 3+ characters)rag_context = ", ".join(set([match['text'] for match in matches])) # matched_entity, textresult_cld = extraction_chain.invoke({"text": msg_text, "schema": str(schema_cld), "rag_context":rag_context})print(json.dumps(result_cld, indent=4, ensure_ascii=False))

{
    "message_info": {
        "title": "[SK텔레콤] ACT대리점 SKT 신현본점에서 꼭꼭 숨겨뒀던 할인 꿀팁 방출",
        "main_theme": "통신비 할인 혜택 안내",
        "period": "5월 한정 프로모션(기가인터넷+TV 요금무료 관련)"
    },
    "purpose": [
        "대리점 방문 유도",
        "할인 혜택 안내",
        "상품 가입 유도"
    ],
    "target": [
        {
            "segment": "루원이편한세상 하늘채아파트 거주자",
            "characteristics": "통신비 절감에 관심 있는 고객",
            "priority": 1
        },
        {
            "segment": "스마트폰 구매 예정 고객",
            "characteristics": "갤럭시 S23 또는 퀀텀3에 관심 있는 고객",
            "priority": 2
        },
        {
            "segment": "인터넷/TV 서비스 가입 고객",
            "characteristics": "하나 원더카드 보유 또는 가입 가능 고객",
            "priority": 3
        }
    ],
    "product": [
        {
            "name": "기가인터넷+TV",
            "category": "통신 서비스",
            "benefit": {
                "name": "요금무료",
                "type": "할인"
            },
            "conditions": "하나 원더카드가 SK인터넷요금 2년간 지원(전월 카드 40만원이상 사용기준)",
           

In [ ]:
for index in range(len(result_cld['product'])):
    item_name = result_cld['product'][index]['name'] #+ " " + result['광고 상품'][0]['상품 정보']
    # get_relevant_context(item_name, vectorstore, topk=20)['title']
    matches = find_entities_in_text(
        result_cld['product'][index]['name'],
        entity_list,
        min_similarity=70,  # Lower threshold for more matches
        ngram_size=3,       # Bigrams work well for Korean
        min_entity_length=3 # Only consider entities with 3+ characters
    )
    schema_res = {
    "ext_item_nm":{"type":"string","description":"기준 item의 name"},
    "item_id":{"type":"string","description":"가장 유사한 item의 id"},
    "item_nm":{"type":"string","description":"가장 유사한 item의 name"},
    "domain":{"type":"string","description":"해당 item의 도메인"},
    "reason":{"type":"string","description":"선택한 이유"}
    }
    prompt = f"""
    아래 광고 메세지에서 추출한 상품 정보와 후보 상품 리스트 중에서 광고 메세지의 맥락에 가장 부합하는 상품을 선택해라.  다음과 같은 schema에 참고하여 JSON 개체로 만들어 주세요:\n {schema_res}
    ---상품 정보---
    상품명: {result_cld['product'][index]['name']}
    카테고리: {result_cld['product'][index]['category']}
    혜택: {result_cld['product'][index]['benefit']}
    조건: {result_cld['product'][index]['conditions']}
    반응: {result_cld['product'][index]['action']}
    ---원본 광고 메세지---
    {msg_text}
    ---후보 상품 리스트---
    {pd.DataFrame(matches)}
    """
    res_cld = extract_json_objects(llm_cld37.invoke(prompt).content)[0]
    print(json.dumps(res_cld, indent=4, ensure_ascii=False))

In [ ]:
prompt = f"""
아래 광고 메세지의 구조 분석해라.
분석의 목적은 메세지에서 구조화된 정보를 추출하기 위함이다.
다양한 구조의 다른 메세지들도 분석할 예정인데, 이를 위한 prompt에 추가할 JSON이 아래와 같은 데, 개선점을 파악해서, 새로운 JSON을 추천해라.
===광고 메세지 예시===
{mdf[:15]}
===PROMPT에 넣을 JSON===
{schema_ax}
"""
response = client.chat.completions.create(
    model="skt/a.x-3-lg",  # or your specific model
    messages=[
        {"role": "user", "content":prompt}
    ],
    temperature=0.0
)
print(response.choices[0].message.content)


In [14]:
schema_ax = {    "message_info": {        "type": "object",        "properties": {            "title": {"type": "string", "description": "광고 제목"},            "main_theme": {"type": "string", "description": "광고의 주요 테마/주제"},            "period": {"type": "string", "description": "이벤트/프로모션 기간"}        }    },    "purpose": {"type": "array", "description": "[상품 가입 유도, 대리점 방문 유도, 웹/앱 접속 유도, 이벤트 응모 유도, 할인 혜택 안내, 쿠폰 제공 안내, 경품 제공 안내, 기타 정보 제공] 중에서 선택. 애매모호한 상황인 경우, 복수 선택 가능."},    "target": {        "type": "array",        "items": {            "type": "object",            "properties": {                "segment": {"type": "string", "description": "타겟 고객층"},                "characteristics": {"type": "string", "description": "해당 타겟의 특성"},                "priority": {"type": "integer", "description": "타겟팅 우선순위"}            }        }    },    "product": {            "type": "array",            "items": {                "type": "object",                "properties": {                    "name": {"type": "string", "description": "광고하는 제품이나 서비스. 광고 메세지에 표현된 원본 그대로 추출해 주세요."},                    "category": {"type": "string", "description": "광고 상품의 카테고리"},                    "benefit": {                        "type": "object",                        "properties": {                            "name": {"type": "string", "description": "제공되는 혜택 이름"},                            "type": {"type": "string", "description": "혜택 타입. [할인, 쿠폰, 경품, 기타] 중에서 선택"}                        }                    },                    "conditions": {"type": "string", "description": "혜택/구매 조건"},                    "action": {"type": "string", "description": "해당 상품에 대해서 고객이 하기 바라는 행동. [구매, 가입, 사용, 방문, 참여, 작성, 기타] 중에서 선택"}                },                "required": ["name", "info"]            }    },    "channel": {        "type": "array",        "items": {            "type": "object",            "properties": {                "type": {"type": "string", "description": "채널의 종류 (URL, 전화번호, 앱, 대리점)"},                "value": {"type": "string", "description": "실제 URL, 전화번호, 앱 이름, 대리점 이름 등의 상세 정보"},                "action": {"type": "string", "description": "채널 제공의 목적. [가입, 추가 정보, 문의, 수신, 수신 거부] 중에서 선택"},                "primary": {"type": "boolean", "description": "주요 채널 여부"},                "availability": {"type": "string", "description": "채널 이용 가능 시간/조건"},                "benefit": {"type": "string", "description": "해당 채널 이용 시 혜택"}            },            "required": ["type", "value"]        }    },    "metadata": {        "type": "object",        "properties": {            "message_type": {"type": "string"},            "target_response": {"type": "string"},            "success_metrics": {"type": "array"}        }    },        "required": ["purpose", "target", "product", "channel","metadata"],    "objectType": "object"}msg_text = """광고 제목: [SK텔레콤] ACT대리점 SKT 신현본점에서 꼭꼭 숨겨뒀던 할인 꿀팁 방출    광고 내용: (광고)[SKT] 고객님 안녕하세요._루원이편한세상 하늘채아파트 정문 신현본점에서_통신비 할인 비책 공개합니다!__① 기가인터넷+TV 요금무료_ 5월 한정 프로모션, 하나 원더카드가_ SK인터넷요금 2년간 지원_ (전월 카드 40만원이상 사용기준)__② S23 할인이 중복_ S23은 약정할인과 제휴카드 할인이 중복된다는 사실!_ 갓성비 갤럭시 퀀텀3도 마찬가지__③ SK공식인증매장에서 더 싸다?_ SK매직/ADT캡스와 제휴하여_ 같은 공기청정기/정수기/홈보안이라도 훨씬 저렴하게 이용!__□ 매장으로 문의주세요 (네이버,매장 홈페이지로 방문예약 가능)__◆ 단골등록 이벤트_ 단골고객 모두에게_ 스마트폰 구매 시 나와 지인 같이 할인받는_ "지인찬스 쿠폰" 4만원권을 발송__■ ACT대리점 신현본점_- 주소: 인천 서구 가정로 387, 가동 201호 SK텔레콤_- 연락처: 0507-1404-2560_- 네이버로 보기: http://t-mms.kr/t.do?m=#61&s=19725&a=&u=https://m.place.naver.com/place/1108219331/home_- 매장 위치보기: http://t-mms.kr/t.do?m=#61&s=19726&a=&u=https://m.tworld.co.kr/customer/agentsearch/detail?code=D13546-0297_- 매장 홈페이지: http://t-mms.kr/t.do?m=#61&s=19727&a=&u=https://tworldfriends.co.kr/D135460297__■ 문의 : SKT고객센터(1558,무료)_무료 수신거부 1504_""".replace('_','\n')# msg_text = remove_control_characters(msg_text)#[-700:]# print(msg_text)similarities = cosine_similarity(            tfidf_desc.transform([msg_text]),            tfidf_matrix_desc        )[0]        # Assign similarities to dataframefew_shot_temp = few_shot_pdf.copy()  # Create a copy to avoid modifying originalfew_shot_temp['sim'] = similaritiesfew_shot_temp['rank'] = few_shot_temp["sim"].rank(method='min', ascending=False)# Get top 5 examplesfew_shot_exm = []top_examples = few_shot_temp[    (few_shot_temp['rank'] >= 2) &     (few_shot_temp['rank'] <= 2)].sort_values("rank")for _, r in top_examples.iterrows():    few_shot_exm.append(f"""[학습용 광고 메세지]    광고 제목:{r['msg_head']}    광고 내용:{r['msg_body']}    [학습용 정답 결과]    {r['res_chat']}""")few_shot_exm_str = "\n".join(few_shot_exm)cot_str = """이 광고는 SK텔레콤 대리점의 복합 프로모션을 안내하는 메시지입니다. 주요 목적은 할인 혜택 안내와 대리점 방문 유도이며, 특정 아파트 단지를 타겟팅하고 있습니다. 통신 서비스, 스마트폰, 가전제품 등 다양한 제품군에 대한 할인 혜택을 제시하며, 여러 채널을 통한 접근 방법을 안내하고 있습니다."""matches = find_entities_in_text(    msg_text,     entity_list,     min_similarity=70,  # Lower threshold for more matches    ngram_size=3,       # Bigrams work well for Korean    min_entity_length=3 # Only consider entities with 3+ characters)rag_context = ", ".join(set([match['matched_entity'] for match in matches])) # matched_entity, text# print(rag_context)# rag_context = rag_context+", "+ ", ".join([r['상품명'] for r in result_cld['광고 상품']])# Create the system and user messagessystem_message_ax = f"당신은 캠페인 메시지에서 정보를 추출하는 전문가입니다. 다음과 같은 schema에 매칭되는 JSON 개체를 만들어 주세요:\n {schema_ax}"# Create the chat completionresponse = client.chat.completions.create(    model="skt/a.x-3-lg",  # or your specific model    messages=[        {"role": "system", "content": system_message_ax},        {"role": "user", "content":          # "\n\n"+system_message_ax+         "\n\n=====분석 대상 광고 메세지=====\n"+msg_text+         "\n\n=====관련 상품 이름=====\n"+rag_context+        # "\n\n=====추론 가이드=====\n"+cot_str+        #  "\n\n아래에 몇 가지 예시들이 있다."        # "\n=====학습용 예시=====\n"+few_shot_exm_str         ""        }    ],    temperature=0.0)# Parse the responseresult_ax = extract_json_objects(response.choices[0].message.content)[0]print(json.dumps(result_ax, indent=4, ensure_ascii=False))

{
    "message_info": {
        "title": "[SK텔레콤] ACT대리점 SKT 신현본점에서 꼭꼭 숨겨뒀던 할인 꿀팁 방출",
        "main_theme": "통신비 할인 및 제휴 혜택",
        "period": "5월 한정 프로모션"
    },
    "purpose": [
        "상품 가입 유도",
        "대리점 방문 유도",
        "웹/앱 접속 유도",
        "이벤트 응모 유도",
        "할인 혜택 안내",
        "쿠폰 제공 안내"
    ],
    "target": [
        {
            "segment": "SK텔레콤 고객",
            "characteristics": "통신비 절약에 관심 있는 고객",
            "priority": "1"
        },
        {
            "segment": "루원이편한세상 하늘채아파트 주민",
            "characteristics": "가까운 대리점을 이용할 가능성이 높은 고객",
            "priority": "2"
        }
    ],
    "product": [
        {
            "name": "기가인터넷+TV 요금무료",
            "category": "통신 서비스",
            "benefit": {
                "name": "통신비 할인",
                "type": "할인"
            },
            "conditions": "하나 원더카드 전월 40만원 이상 사용 기준",
            "action": "가입"
        },
        {
            "name": "S23 할인",
            "category": "스마트폰",
            "ben

In [ ]:
# schema_ax = {
#         "purpose": {"type": "array", "description": "[상품 가입 유도, 대리점 방문 유도, 웹/앱 접속 유도, 이벤트 응모 유도, 할인 혜택 안내, 쿠폰 제공 안내, 경품 제공 안내, 기타 정보 제공] 중에서 선택."},
#         "target": {"type": "array", "description": "광고가 타겟팅하는 고객층"},
#         "product": {
#             "type": "array",
#             "items": {
#                 "type": "object",
#                 "properties": {
#                     "name": {"type": "string", "description": "광고하는 제품이나 서비스. 광고 메세지에 표현된 원본 그대로 추출해 주세요."},
#                     "category": {"type": "string", "description": "광고 상품의 카테고리"},
#                     "info": {"type": "string", "description": "광고 상품에 대한 정보"},
#                     "action": {"type": "string", "description": "해당 상품에 대해서 고객이 하기 바라는 행동. [구매, 가입, 사용, 방문, 참여, 작성, 기타] 중에서 선택"}
#                 },
#                 "required": ["name", "info"]
#             }
#         },
#         "channel": {
#             "type": "array",
#             "items": {
#                 "type": "object",
#                 "properties": {
#                     "type": {"type": "string", "description": "채널의 종류 (URL, 전화번호, 앱, 대리점)"},
#                     "value": {"type": "string", "description": "실제 URL, 전화번호, 앱 이름, 대리점 이름 등의 상세 정보"},
#                     "action": {"type": "string", "description": "채널 제공의 목적. [가입, 추가 정보, 문의, 수신] 중에서 선택"}
#                 },
#                 "required": ["type", "value"]
#             }
#         }
# }
msg_text = """
광고 제목: [SK텔레콤] ACT대리점 SKT 신현본점에서 꼭꼭 숨겨뒀던 할인 꿀팁 방출
광고 내용: (광고)[SKT] 고객님 안녕하세요._루원이편한세상 하늘채아파트 정문 신현본점에서_통신비 할인 비책 공개합니다!__① 기가인터넷+TV 요금무료_ 5월 한정 프로모션, 하나 원더카드가_ SK인터넷요금 2년간 지원_ (전월 카드 40만원이상 사용기준)__② S23 할인이 중복_ S23은 약정할인과 제휴카드 할인이 중복된다는 사실!_ 갓성비 갤럭시 퀀텀3도 마찬가지__③ SK공식인증매장에서 더 싸다?_ SK매직/ADT캡스와 제휴하여_ 같은 공기청정기/정수기/홈보안이라도 훨씬 저렴하게 이용!__□ 매장으로 문의주세요 (네이버,매장 홈페이지로 방문예약 가능)__◆ 단골등록 이벤트_ 단골고객 모두에게_ 스마트폰 구매 시 나와 지인 같이 할인받는_ "지인찬스 쿠폰" 4만원권을 발송__■ ACT대리점 신현본점_- 주소: 인천 서구 가정로 387, 가동 201호 SK텔레콤_- 연락처: 0507-1404-2560_- 네이버로 보기: http://t-mms.kr/t.do?m=#61&s=19725&a=&u=https://m.place.naver.com/place/1108219331/home_- 매장 위치보기: http://t-mms.kr/t.do?m=#61&s=19726&a=&u=https://m.tworld.co.kr/customer/agentsearch/detail?code=D13546-0297_- 매장 홈페이지: http://t-mms.kr/t.do?m=#61&s=19727&a=&u=https://tworldfriends.co.kr/D135460297__■ 문의 : SKT고객센터(1558,무료)_무료 수신거부 1504_
""".replace('_','\n')
matches = find_entities_in_text(
    msg_text,
    entity_list,
    min_similarity=70,  # Lower threshold for more matches
    ngram_size=3,       # Bigrams work well for Korean
    min_entity_length=3 # Only consider entities with 3+ characters
)
rag_context = ", ".join(set([match['text'] for match in matches])) # matched_entity, text
prompt_list = [{"schema":schema_ax,"msg_text": "=====분석 대상 광고 메세지=====\n"+msg_text+"\n=====후보 상품 이름=====\n"+rag_context}, {"schema":{"channel":schema_ax['channel']},"msg_text":msg_text}]

In [ ]:
def process_prompt(row):
    schema_ax = row['schema']
    msg_text = row['msg_text']
    system_message_ax = f"당신은 캠페인 메시지에서 정보를 추출하는 전문가입니다. 다음과 같은 schema에 부합하도록 JSON 개체를 만들어 주세요:\n {schema_ax}"
    # Create the chat completion
    response = client.chat.completions.create(
        model="skt/a.x-3-lg",  # or your specific model
        messages=[
            {"role": "system", "content": system_message_ax},
            {"role": "user", "content": msg_text}
        ],
        temperature=0.0
    )
    # print(response.choices[0].message.content)
    # Parse the response
    result_ax = extract_json_objects(response.choices[0].message.content)[0]
    return result_ax
results_ax_mul = []
with ThreadPoolExecutor(max_workers=len(prompt_list)) as executor:  # More threads for I/O-bound tasks
    results_ax_mul += list(executor.map(process_prompt, prompt_list))
result_ax = {}
for key in list(set(sum([list(d.keys()) for d in results_ax_mul], []))):
    lst = []
    for dic in results_ax_mul:
        if key in dic:
            lst.append(dic[key])
    dict_list = sum(lst,[])
    if is_list_of_dicts(dict_list):
        df = pd.DataFrame(dict_list)
        df = df.drop_duplicates()
        dict_list = df.to_dict('records')
    result_ax[key] = dict_list
print(json.dumps(result_ax, indent=4, ensure_ascii=False))


In [ ]:
for index in range(len(result_ax['product'])):
    item_name = result_ax['product'][index]['name'] #+ " " + result['광고 상품'][0]['상품 정보']
    # get_relevant_context(item_name, vectorstore, topk=20)['title']
    # item_pdf['sim'] = cosine_similarity(
    #     tfidf_name.transform([result_ax['광고 상품'][index]['상품명'] ]),
    #     tfidf_matrix_name
    # )[0]
    matches = find_entities_in_text(
        result_ax['product'][index]['name'],
        entity_list,
        min_similarity=70,  # Lower threshold for more matches
        ngram_size=3,       # Bigrams work well for Korean
        min_entity_length=3 # Only consider entities with 3+ characters
    )
    schema_res = {
    "ext_item_nm":{"type":"string","description":"기준 item의 name"},
    "item_id":{"type":"string","description":"가장 유사한 item의 id"},
    "item_nm":{"type":"string","description":"가장 유사한 item의 name"},
    "domain":{"type":"string","description":"해당 item의 도메인"},
    "reason":{"type":"string","description":"선택한 이유"}
    }
    text = f"""
    ---상품 정보---
    상품명: {result_ax['product'][index]['name']}
    상품 카테고리: {result_ax['product'][index]['category']}
    상품 정보: {result_ax['product'][index]['info']}
    ---원본 광고 메세지---
    {msg_text}
    ---후보 상품 리스트---
    {pd.DataFrame(matches)}
    """
    system_message_ax = f"""
    아래 광고 메세지에서 추출한 상품 정보와 후보 상품 리스트 중에서 광고 메세지의 맥락에 가장 부합하는 상품을 선택해라. 다음과 같은 schema에 참고하여 JSON 개체로 만들어 주세요:\n {schema_res}
    """
    # Create the chat completion
    response = client.chat.completions.create(
        model="skt/a.x-3-lg",  # or your specific model
        messages=[
            {"role": "system", "content": system_message_ax},
            {"role": "user", "content": text}
        ],
        temperature=0
    )
    res_ax = extract_json_objects((response.choices[0].message.content).replace("'",'"'))[0]
    print(json.dumps(res_ax, indent=4, ensure_ascii=False))

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from collections import Counter
import konlpy.tag
# For Korean text preprocessing, use KoNLPy
class KoreanTokenizer:
    def __init__(self, tagger_type='Okt'):
        # Choose a Korean morphological analyzer
        # Options: Okt, Mecab, Komoran, Hannanum, Kkma
        if tagger_type == 'Okt':
            from konlpy.tag import Okt
            self.tagger = Okt()
        elif tagger_type == 'Mecab':
            from konlpy.tag import Mecab
            self.tagger = Mecab()
        # Add other taggers as needed
    def __call__(self, text):
        # Extract nouns, adjectives, and verbs (you can customize this)
        tokens = self.tagger.morphs(text)
        return tokens
# Initialize the tokenizer (Okt is generally good for most purposes)
korean_tokenizer = KoreanTokenizer(tagger_type='Okt')
tfidf_name = TfidfVectorizer(
            analyzer='char',
            ngram_range=(3,10),
            min_df=1,
            # stop_words=None,  # Don't use stop words initially
            # token_pattern=r'(?u)\b\w+\b',  # Match any word character (more permissive)
            # strip_accents='unicode',  # Handle accented characters
        )
tfidf_cate = TfidfVectorizer(
            analyzer='char',
            ngram_range=(2,4),
            min_df=1,
            # stop_words=None,  # Don't use stop words initially
            # token_pattern=r'(?u)\b\w+\b',  # Match any word character (more permissive)
            # strip_accents='unicode',  # Handle accented characters
        )
tfidf_desc = TfidfVectorizer(
            analyzer='char',
            ngram_range=(3,5),
            min_df=1,
            # stop_words=None,  # Don't use stop words initially
            # token_pattern=r'(?u)\b\w+\b',  # Match any word character (more permissive)
            # strip_accents='unicode',  # Handle accented characters
        )
# tfidf_desc = TfidfVectorizer(
#             analyzer='word',
#             ngram_range=(1,5),
#             min_df=1,
#             stop_words=None,  # Don't use stop words initially
#             token_pattern=r'(?u)\b\w+\b',  # Match any word character (more permissive)
#             strip_accents='unicode',  # Handle accented characters
#         )
# tfidf_name = TfidfVectorizer(
#     tokenizer=korean_tokenizer,
#     ngram_range=(1, 2),  # Consider both unigrams and bigrams
#     max_features=10000,  # Limit vocabulary size
#     min_df=2,            # Minimum document frequency
#     use_idf=True,
#     sublinear_tf=True    # Apply sublinear tf scaling (1 + log(tf))
# )
# tfidf_desc = TfidfVectorizer(
#     tokenizer=korean_tokenizer,
#     ngram_range=(1, 2),  # Consider both unigrams and bigrams
#     max_features=10000,  # Limit vocabulary size
#     min_df=2,            # Minimum document frequency
#     use_idf=True,
#     sublinear_tf=True    # Apply sublinear tf scaling (1 + log(tf))
# )
# tfidf_item = TfidfVectorizer(
#     tokenizer=korean_tokenizer,
#     ngram_range=(1, 2),  # Consider both unigrams and bigrams
#     max_features=10000,  # Limit vocabulary size
#     min_df=2,            # Minimum document frequency
#     use_idf=True,
#     sublinear_tf=True    # Apply sublinear tf scaling (1 + log(tf))
# )
few_shot_pdf = pd.read_pickle("./data/few_shot_data_chat_250410_rd_500.pkl")
few_shot_pdf = few_shot_pdf.astype('str')
tfidf_matrix_name = tfidf_name.fit_transform(item_pdf['item_nm_cl'])
tfidf_matrix_cate = tfidf_cate.fit_transform(item_pdf['item_cate_ax'])
tfidf_matrix_desc = tfidf_desc.fit_transform(few_shot_pdf["msg_body"])
# tfidf_matrix_item = tfidf_item.fit_transform(item_pdf['item_item_cl'])
# import torch
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# emb_model = SentenceTransformer('snunlp/KR-SBERT-V40K-klueNLI-augSTS')  # Or other appropriate model
# emb_model = emb_model.to(device)
# item_all = item_pdf['item_item_cl'].tolist()
# item_embeddings = emb_model.encode(item_all)

In [ ]:
from langchain_anthropic import ChatAnthropic
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
# Define your schema
schema_cld = {
    "properties": {
        "message_info": {
            "type": "object",
            "properties": {
                "title": {"type": "string", "description": "광고 제목"},
                "main_theme": {"type": "string", "description": "광고의 주요 테마/주제"},
                "period": {"type": "string", "description": "이벤트/프로모션 기간"}
            }
        },
        "purpose": {"type": "array", "description": "[상품 가입 유도, 대리점 방문 유도, 웹/앱 접속 유도, 이벤트 응모 유도, 할인 혜택 안내, 쿠폰 제공 안내, 경품 제공 안내, 기타 정보 제공] 중에서 선택. 애매모호한 상황인 경우, 복수 선택 가능."},
        "target": {
            "type": "array",
            "items": {
                "type": "object",
                "properties": {
                    "segment": {"type": "string", "description": "타겟 고객층"},
                    "characteristics": {"type": "string", "description": "해당 타겟의 특성"},
                    "priority": {"type": "integer", "description": "타겟팅 우선순위"}
                }
            }
        },
        "product": {
            "type": "array",
            "items": {
                "type": "object",
                "properties": {
                    "name": {"type": "string", "description": "광고하는 제품이나 서비스. 광고 메세지에 표현된 원본 그대로 추출해 주세요."},
                    "category": {"type": "string", "description": "광고 상품의 카테고리"},
                    "benefit": {
                        "type": "object",
                        "properties": {
                            "name": {"type": "string", "description": "제공되는 혜택 이름"},
                            "type": {"type": "string", "description": "혜택 타입. [할인, 쿠폰, 경품, 기타] 중에서 선택"}
                        }
                    },
                    "conditions": {"type": "string", "description": "혜택/구매 조건"},
                    "action": {"type": "string", "description": "해당 상품에 대해서 고객이 하기 바라는 행동. [구매, 가입, 사용, 방문, 참여, 작성, 기타] 중에서 선택"}
                },
                "required": ["name", "info"]
            }
        },
        "channel": {
            "type": "array",
            "items": {
                "type": "object",
                "properties": {
                    "type": {"type": "string", "description": "채널의 종류 (URL, 전화번호, 앱, 대리점)"},
                    "value": {"type": "string", "description": "실제 URL, 전화번호, 앱 이름, 대리점 이름 등의 상세 정보"},
                    "action": {"type": "string", "description": "채널 제공의 목적. [가입, 추가 정보, 문의, 수신, 수신 거부] 중에서 선택"},
                    "primary": {"type": "boolean", "description": "주요 채널 여부"},
                    "availability": {"type": "string", "description": "채널 이용 가능 시간/조건"},
                    "benefit": {"type": "string", "description": "해당 채널 이용 시 혜택"}
                },
                "required": ["type", "value"]
            }
        }
    },
    "metadata": {
        "type": "object",
        "properties": {
            "message_type": {"type": "string"},
            "target_response": {"type": "string"},
            "success_metrics": {"type": "array"}
        }
    },
    "required": ["purpose", "target", "product", "channel","metadata"],
    "objectType": "object"
}
# Create a prompt template
prompt_cld = ChatPromptTemplate.from_messages([
    ("system", "당신은 캠페인 메시지에서 정보를 추출하는 전문가입니다. 다음과 같은 schema에 매칭되는 JSON 개체를 만들어 주세요:\n {schema}"
     # "\n\n제품 참조 정보:\n{rag_context}"
    ),
    ("human", "{text}")
])
schema_ax = {
    "message_info": {
        "type": "object",
        "properties": {
            "title": {"type": "string", "description": "광고 제목"},
            "main_theme": {"type": "string", "description": "광고의 주요 테마/주제"},
            "period": {"type": "string", "description": "이벤트/프로모션 기간"}
        }
    },
    "purpose": {"type": "array", "description": "[상품 가입 유도, 대리점 방문 유도, 웹/앱 접속 유도, 이벤트 응모 유도, 할인 혜택 안내, 쿠폰 제공 안내, 경품 제공 안내, 기타 정보 제공] 중에서 선택. 애매모호한 상황인 경우, 복수 선택 가능."},
    "target": {
        "type": "array",
        "items": {
            "type": "object",
            "properties": {
                "segment": {"type": "string", "description": "타겟 고객층"},
                "characteristics": {"type": "string", "description": "해당 타겟의 특성"},
                "priority": {"type": "integer", "description": "타겟팅 우선순위"}
            }
        }
    },
    "product": {
            "type": "array",
            "items": {
                "type": "object",
                "properties": {
                    "name": {"type": "string", "description": "광고하는 제품이나 서비스. 광고 메세지에 표현된 원본 그대로 추출해 주세요."},
                    "category": {"type": "string", "description": "광고 상품의 카테고리"},
                    "benefit": {
                        "type": "object",
                        "properties": {
                            "name": {"type": "string", "description": "제공되는 혜택 이름"},
                            "type": {"type": "string", "description": "혜택 타입. [할인, 쿠폰, 경품, 기타] 중에서 선택"}
                        }
                    },
                    "conditions": {"type": "string", "description": "혜택/구매 조건"},
                    "action": {"type": "string", "description": "해당 상품에 대해서 고객이 하기 바라는 행동. [구매, 가입, 사용, 방문, 참여, 작성, 기타] 중에서 선택"}
                },
                "required": ["name", "info"]
            }
    },
    "channel": {
        "type": "array",
        "items": {
            "type": "object",
            "properties": {
                "type": {"type": "string", "description": "채널의 종류 (URL, 전화번호, 앱, 대리점)"},
                "value": {"type": "string", "description": "실제 URL, 전화번호, 앱 이름, 대리점 이름 등의 상세 정보"},
                "action": {"type": "string", "description": "채널 제공의 목적. [가입, 추가 정보, 문의, 수신, 수신 거부] 중에서 선택"},
                "primary": {"type": "boolean", "description": "주요 채널 여부"},
                "availability": {"type": "string", "description": "채널 이용 가능 시간/조건"},
                "benefit": {"type": "string", "description": "해당 채널 이용 시 혜택"}
            },
            "required": ["type", "value"]
        }
    },
    "metadata": {
        "type": "object",
        "properties": {
            "message_type": {"type": "string"},
            "target_response": {"type": "string"},
            "success_metrics": {"type": "array"}
        }
    },
    "required": ["purpose", "target", "product", "channel","metadata"],
    "objectType": "object"
}


In [15]:
def get_llm_result_by_fsl(msg_text, rag_context, few_shot_temp_temp, res_dic_empt, row, fs_model, num_fs=1):
    # Get top 5 examples
    few_shot_exm = []
    top_examples = few_shot_temp_temp[
        (few_shot_temp_temp['rank'] >= 1) &
        (few_shot_temp_temp['rank'] <= num_fs)
    ].sort_values("rank")
    for _, r in top_examples.iterrows():
        few_shot_exm.append(f"""
[학습용 광고 메세지]
광고 제목:{r['msg_head']}
광고 내용:{r['msg_body']}
[학습용 정답 결과]
{r[f'res_{fs_model}']}
        """)
        row[f"""fsl_head_{int(r['rank'])}"""] = r['msg_head']
        row[f"""fsl_body_{int(r['rank'])}"""] = r['msg_body']
    few_shot_exm_str = "\n".join(few_shot_exm)
    # print(few_shot_exm_str)
    try:
        system_message_ax = f"위 예시들을 참고하여, 다음과 같은 schema에 부합하도록 JSON 개체만을 만들어 주세요.:\n {schema_ax}"
        prompt = (
"광고 메세지에서 정보들을 추출하고자 한다. 아래에 몇 가지 예시들이 있다."
"\n\n=====학습용 예시=====\n"+few_shot_exm_str+
"\n\n"+system_message_ax+
"\n\n=====분석 대상 광고 메세지=====\n"+msg_text
+"\n=====후보 상품 이름=====\n"+rag_context
                )
        response = client.chat.completions.create(
            model="skt/a.x-3-lg",  # or your specific model
            messages=[
                # {"role": "system", "content": system_message_ax},
                {"role": "user", "content": prompt
                }
            ],
            temperature=0.0
        )
        result_raw = response.choices[0].message.content
        result = extract_json_objects(result_raw)[0]
        row[f'res_ax_f{num_fs}'] = result
    except Exception as e:
        print(f"Error message when calling ax_f{num_fs}: \n{result_raw}")
        row[f'res_ax_f{num_fs}'] = res_dic_empt#.replace('\n','')
    return row
def get_llm_result_by_fsl_exp(msg_text, rag_context, few_shot_temp_temp, res_dic_empt, row, fs_model, num_fs=1):
    # Get top 5 examples
    few_shot_exm = []
    top_examples = few_shot_temp_temp[
        (few_shot_temp_temp['rank'] >= 1) &
        (few_shot_temp_temp['rank'] <= num_fs)
    ].sort_values("rank")
    for _, r in top_examples.iterrows():
        few_shot_exm.append(f"""
[학습용 광고 메세지]
광고 제목:{r['msg_head']}
광고 내용:{r['msg_body']}
[학습용 정답 결과]
{r[f'res_{fs_model}']}
[학습용 Chain-of-Thought]
{r[f'cot_{fs_model}']}
        """)
        row[f"""fsl_head_{int(r['rank'])}"""] = r['msg_head']
        row[f"""fsl_body_{int(r['rank'])}"""] = r['msg_body']
    few_shot_exm_str = "\n".join(few_shot_exm)
    # print(few_shot_exm_str)
    try:
        system_message_ax = f"위 예시들을 참고하여, 다음과 같은 schema에 부합하도록 JSON 개체만을 만들어 주세요.:\n {schema_ax}"
        prompt = (
"광고 메세지에서 정보들을 추출하고자 한다. 아래에 몇 가지 예시들과 Chain-of-Thought가 있다."
"\n\n=====학습용 예시=====\n"+few_shot_exm_str+
"\n\n"+system_message_ax+
"\n\n=====분석 대상 광고 메세지=====\n"+msg_text
+"\n=====후보 상품 이름=====\n"+rag_context
                )
        response = client.chat.completions.create(
            model="skt/a.x-3-lg",  # or your specific model
            messages=[
                # {"role": "system", "content": system_message_ax},
                {"role": "user", "content": prompt
                }
            ],
            temperature=0.0
        )
        result_raw = response.choices[0].message.content
        result = extract_json_objects(result_raw)[0]
        row[f'res_ax_f{num_fs}_x'] = result
    except Exception as e:
        print(f"Error message when calling ax_f{num_fs}_x: \n{result_raw}")
        row[f'res_ax_f{num_fs}'] = res_dic_empt#.replace('\n','')
    return row
                        

In [ ]:
import pandas as pdimport jsonbuild_few_shot = Trueref_model = "cld35"if ref_model == "cld35":    llm_modl = llm_cld35elif ref_model == "cld37":    llm_modl = llm_cld37else:    llm_modl = llm_chatn_sample_fsl = 500n_fsl_sta = 1n_fsl_end = 1# Create the parserparser = JsonOutputParser()extraction_chain = prompt_cld | llm_modl | parserextraction_chain_cld35 = prompt_cld | llm_cld35 | parser# few_shot_pdf = pd.read_pickle(f"/home/skinet/myfiles/tos_ace/data/mms_extraction_data_cld_fsl_250404_rd_{n_sample_fsl}.pkl").iloc[:100]rep_dict = {'[SK텔레콤]':'','[SKT]':''}special_symbols_to_remove = {'‘':'','’':'',':':'',"'":'','"':'','_':'\n'}def process_prompt(row, few_shot_pdf, ref_model):    res_dic_empt = {        "purpose": "",        "target": "",        "product": "",        "channel": ""    }        head = row['head']    body = row['body']        head = replace_strings(head, special_symbols_to_remove)    body = replace_strings(body, special_symbols_to_remove)            #     for k,v in special_symbols_to_remove.items():#         head = head.replace(k,v)#         body = body.replace(k,v)        msg_text = f"""        광고 제목: {head}        광고 내용: {body}    """        #     rag_context = get_relevant_context(text, vectorstore, topk=20)['title']    #     row['res_rag'] = rag_context    matches = find_entities_in_text(        msg_text,         entity_list,         min_similarity=70,  # Lower threshold for more matches        ngram_size=3,       # Bigrams work well for Korean        min_entity_length=3 # Only consider entities with 3+ characters    )        rag_context = ", ".join(set([match['text'] for match in matches])) # matched_entity, text        if not build_few_shot:        # Calculate similarities        similarities = cosine_similarity(            tfidf_desc.transform([msg_text]),            tfidf_matrix_desc        )[0]                # Assign similarities to dataframe        few_shot_temp = few_shot_pdf.copy()  # Create a copy to avoid modifying original        few_shot_temp['sim'] = similarities                few_shot_temp_temp = few_shot_temp.query(f"offer_dt<'{row['offer_dt']}'").copy()        few_shot_temp_temp['rank'] = few_shot_temp_temp["sim"].rank(method='min', ascending=False)                for nf in range(n_fsl_sta, n_fsl_end+1):            row = get_llm_result_by_fsl(msg_text, rag_context, few_shot_temp_temp, res_dic_empt, row, ref_model, num_fs=nf)        # for nf in range(n_fsl_sta, n_fsl_end+1):        #     row = get_llm_result_by_fsl_exp(msg_text, rag_context, few_shot_temp_temp, res_dic_empt, row, ref_model, num_fs=nf)        # try:        #     response = client.chat.completions.create(        #         model="skt/a.x-3-lg",  # or your specific model        #         messages=[        #             {"role": "system", "content": f"당신은 캠페인 메시지에서 정보를 추출하는 전문가입니다. 다음과 같은 schema를 참고하여 JSON 개체를 만들어 주세요:\n {schema_ax}"},        #             {"role": "user", "content": msg_text        #              +"\n---후보 상품 이름----\n"+rag_context        #             }        #         ],        #         temperature=0        #     )                    #     result_raw = response.choices[0].message.content        #     result = extract_json_objects(result_raw)[0]        #     # result = json.dumps(result, indent=4, ensure_ascii=False)        #     row['res_ax_f0'] = result        # except Exception as e:        #     print(f"Error message when calling ax_n: \n{result_raw}")        #     row['res_ax_f0'] = res_dic_empt#.replace('\n','')        try:        result = extraction_chain.invoke({"text": msg_text, "schema": str(schema_cld)                                          # , "rag_context":rag_context                                         })        # result = json.dumps(result, indent=4, ensure_ascii=False)        row[f'res_{ref_model}'] = result            except Exception as e:        try:            result = extract_json_objects(e)[0]            row[f'res_{ref_model}'] = result        except Exception as e2:            print(f"Error message when calling {ref_model}: \n{e2}")            row[f'res_{ref_model}'] = res_dic_empt    # try:    #     result = extraction_chain_cld35.invoke({"text": msg_text, "schema": str(schema_cld)    #                                       # , "rag_context":rag_context    #                                      })    #     # result = json.dumps(result, indent=4, ensure_ascii=False)    #     row[f'res_cld35'] = result            # except Exception as e:    #     try:    #         result = extract_json_objects(e)[0]    #         row[f'res_cld35'] = result    #     except Exception as e2:    #         print(f"Error message when calling cld35: \n{e2}")    #         row[f'res_cld35'] = res_dic_empt    return rowstart_time = time.time()import randomfrom functools import partialtest_pdf_list = []# mdf = mms_pdf.rename(columns={'msg_nm':'head','mms_phrs':'body'}).to_dict("records")if build_few_shot:    mdf = mms_pdf.rename(columns={'msg_nm':'head','mms_phrs':'body'}).sample(n=n_sample_fsl).to_dict("records")else:    mdf = mms_pdf.query("offer_dt>='20240101'").merge(few_shot_pdf[['index',f'res_{ref_model}']],on='index',how='left').query(f"res_{ref_model}.isna()").drop(f"res_{ref_model}",axis=1).rename(columns={'msg_nm':'head','mms_phrs':'body'})    mdf = mdf.sample(n=100).to_dict("records")# Create a partial function with few_shot_pdfprocess_prompt_with_pdf = partial(process_prompt, few_shot_pdf=few_shot_pdf, ref_model=ref_model)with ThreadPoolExecutor(max_workers=10) as executor:    test_pdf_list += list(executor.map(process_prompt_with_pdf, mdf))end_time = time.time()elapsed_time = end_time - start_timeprint(f"Execution took {elapsed_time:.4f} seconds")    # test_pdf = pd.DataFrame(test_pdf_list)

In [ ]:
pd.set_option('display.max_colwidth', 1000)if build_few_shot:    rdf = (pd.DataFrame(test_pdf_list).rename(columns={'head':'msg_head','body':'msg_body'}).drop(['msg'], axis=1))    rdf.to_pickle(f"./data/few_shot_data_chat_250409_rd_{n_sample_fsl}.pkl")rdf = (pd.DataFrame(test_pdf_list).rename(columns={'head':'msg_head','body':'msg_body',f'res_ax_f{n_fsl_end}':'res_ax'}).drop(['msg'], axis=1))rdf['msg_body'] = rdf['msg_body'].apply(remove_control_characters)rdf = rdf[(rdf[f'res_ax'].apply(lambda x: x.get('purpose') != '')) & (rdf[f"res_{ref_model}"].apply(lambda x: x.get('purpose') != ''))]rdf['res_ax'] = rdf['res_ax'].apply(lambda x:json.dumps(x, indent=4, ensure_ascii=False))rdf['res_cld'] = rdf['res_cld35'].apply(lambda x:json.dumps(x, indent=4, ensure_ascii=False))rdf[f"res_{ref_model}"] = rdf[f"res_{ref_model}"].apply(lambda x:json.dumps(x, indent=4, ensure_ascii=False))# rdf['res_ax'] = rdf['res_ax'].apply(lambda x: json_to_custom_string(x))# rdf[f"res_{ref_model}"] = rdf[f"res_{ref_model}"].apply(lambda x: json_to_custom_string(x))rdf[['offer_dt','msg_head','msg_body','res_ax','res_cld',f"res_{ref_model}"]].sort_values('offer_dt').to_excel("./data/mms_extraction_data_250410.xlsx", index=False, engine='openpyxl')# row = rdf.to_dict("records")[0]    # rdf['res_ax'].apply(print)# rdf[rdf[f'res_ax_f{n_fsl_end}'].apply(lambda x: x.get('purpose') == '')].shape

In [ ]:
rdf = (pd.DataFrame(test_pdf_list).rename(columns={'head':'msg_head','body':'msg_body'}).drop(['msg'], axis=1))
for nf in (list(range(n_fsl_sta, n_fsl_end+1))):
    rdf = rdf[(rdf[f'res_ax_f{nf}'].apply(lambda x: x.get('purpose') != ''))]
    # rdf = rdf[(rdf['res_ax_f'].apply(lambda x: x.get('purpose') != '')) & (rdf['res_ax_n'].apply(lambda x: x.get('purpose') != ''))]
sdf = rdf.copy()
# # sdf = pd.read_pickle("/home/skinet/myfiles/tos_ace/data/mms_extraction_data_vl_250403_01.pkl")
for nf in (list(range(n_fsl_sta, n_fsl_end+1))):
    sdf[f'sim_f{nf}'] = sdf.apply(lambda x : calculate_dictionary_similarity(x[f'res_ax_f{nf}'], x[f'res_{ref_model}']), axis=1)
    # sdf[f'sim_f{nf}'] = sdf.apply(lambda x : calculate_dictionary_similarity(x[f'res_ax_f{nf}'], x[f'res_cld35']), axis=1)
    print(nf, "\n", pd.json_normalize(sdf[f'sim_f{nf}']).mean())


In [ ]:
def json_to_custom_string(json_obj, indent=0):    result = []    indent_str = "  " * indent        if isinstance(json_obj, dict):        for key, value in json_obj.items():            if isinstance(value, (dict, list)):                result.append(f"{indent_str}{key}:")                result.append(json_to_custom_string(value, indent + 1))            else:                result.append(f"{indent_str}{key}: {value}")    elif isinstance(json_obj, list):        for item in json_obj:            if isinstance(item, (dict, list)):                result.append(json_to_custom_string(item, indent))            else:                result.append(f"{indent_str}- {item}")        return "\n".join(result)# json_to_custom_string(parsed_json)